In [1]:
import cgmquantify as cgm
import pandas as pd
import datetime as datetime
from datetime import date
from scipy.stats import stats
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from scipy.signal import find_peaks
from scipy.stats import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import pickle

 # Evaluation of Parofastin data

In [5]:
parofastin_168 = pd.read_csv('prediction_file_path', sep='\t')
parofastin_bahai = pd.read_csv('prediction_file_path', sep='\t')

In [6]:
fasting_states_to_keep = ['fasting', 'non-fasting']
parofastin_168 = parofastin_168[parofastin_168.fasting_state.isin(fasting_states_to_keep)]
parofastin_bahai = parofastin_bahai[parofastin_bahai.fasting_state.isin(fasting_states_to_keep)]

## Preprocessing for ML models

In [7]:
parofastin_168['Time'] = pd.to_datetime(parofastin_168['time'], format='%d-%m-%Y %H:%M')
parofastin_168['Day'] = parofastin_168["Time"].dt.date#
parofastin_168['gl'] = parofastin_168['gl'].str.replace(',','.')
parofastin_168['gl'] = pd.to_numeric(parofastin_168['gl'])
parofastin_168["gl"] = parofastin_168["gl"]*18
parofastin_168["Glucose"] = parofastin_168["gl"]

parofastin_168 = parofastin_168.sort_values(by=['id', 'time', "phase"], ascending=True)
parofastin_168 = parofastin_168.reset_index(drop=True)
parofastin_168 = parofastin_168.drop(["Unnamed: 0"], axis=1)

parofastin_bahai['Time'] = pd.to_datetime(parofastin_bahai['time'], format='%d-%m-%Y %H:%M')
parofastin_bahai['Day'] = parofastin_bahai["Time"].dt.date#
parofastin_bahai['gl'] = parofastin_bahai['gl'].str.replace(',','.')
parofastin_bahai['gl'] = pd.to_numeric(parofastin_bahai['gl'])
parofastin_bahai["gl"] = parofastin_bahai["gl"]*18
parofastin_bahai["Glucose"] = parofastin_bahai["gl"]
parofastin_bahai = parofastin_bahai.sort_values(by=['id', 'time', "phase"], ascending=True)
parofastin_bahai = parofastin_bahai.reset_index(drop=True)
parofastin_bahai = parofastin_bahai.drop(["Unnamed: 0"], axis=1)

In [2]:
def feat_statistical_measures_gl(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        time_median = df_temp.loc[1,'Time']
        cgm_summary = list(cgm.summary(df_temp))
        cgm_LBGI = cgm.LBGI(df_temp)
        cgm_HBGI = cgm.HBGI(df_temp)
        cgm_ADRR = cgm.ADRR(df_temp)
        cgm_GMI = cgm.GMI(df_temp)
        cgm_J_index = cgm.J_index(df_temp)
        cgm_eA1c = cgm.eA1c(df_temp)
        cgm_interdaysd = cgm.interdaysd(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)
        cgm_TOR = cgm.TOR(df_temp, sd=1, sr=15)
        cgm_TIR = cgm.TIR(df_temp, sd=1, sr=15)
        cgm_POR = cgm.POR(df_temp, sd=1, sr=15)

        
        cgm_summary.append(cgm_LBGI)
        cgm_summary.append(cgm_HBGI)
        cgm_summary.append(cgm_ADRR)
        cgm_summary.append(cgm_GMI)
        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_eA1c)
        cgm_summary.append(cgm_interdaysd)
        cgm_summary.append(cgm_interdaycv)
        cgm_summary.append(cgm_TOR)
        cgm_summary.append(cgm_TIR)
        cgm_summary.append(cgm_POR)

        general = []
        general.extend([time_median])
        general.extend(cgm_summary)

        X_train_temp = pd.DataFrame([general], columns = ["time","mean", "median","minimum", "maximum","first_quartile", "third_quartile",
                                                              "LBGI", "HBGI", "ADRR","GMI", "J_index", "eA1c", "interdaysd",
                                                              "cgm_interdaycv", 
                                                              "cgm_TOR", "cgm_TIR",
                                                              "cgm_POR"])

        
        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [9]:
x_list_parofastin_168 = []
train_labels_parofastin_168 = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, parofastin_168.shape[0] - window_size, step_size):
    time = parofastin_168['Time'].values[i: i + window_size]
    times = parofastin_168['Day'].values[i: i + window_size]
    gls = parofastin_168['Glucose'].values[i: i + window_size]

    label = stats.mode(parofastin_168['fasting_state'][i: i + window_size])[0][0]

    df_slice = pd.DataFrame(time, columns=['Time'])
    df_slice["Day"] = times
    df_slice["Glucose"] = gls   

    x_list_parofastin_168.append(df_slice)

    train_labels_parofastin_168.append(label)

In [10]:
x_list_parofastin_bahai = []
train_labels_parofastin_bahai = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, parofastin_bahai.shape[0] - window_size, step_size):
    time = parofastin_bahai['Time'].values[i: i + window_size]
    times = parofastin_bahai['Day'].values[i: i + window_size]
    gls = parofastin_bahai['Glucose'].values[i: i + window_size]

    label = stats.mode(parofastin_bahai['fasting_state'][i: i + window_size])[0][0]

    df_slice = pd.DataFrame(time, columns=['Time'])
    df_slice["Day"] = times
    df_slice["Glucose"] = gls   

    x_list_parofastin_bahai.append(df_slice)

    train_labels_parofastin_bahai.append(label)

In [11]:
X_pred_parofastin_168 = feat_statistical_measures_gl(x_list_parofastin_168)
X_pred_parofastin_bahai = feat_statistical_measures_gl(x_list_parofastin_bahai)

In [12]:
y_test_168 = np.array(train_labels_parofastin_168)
y_test_bahai = np.array(train_labels_parofastin_bahai)

def to_float(input_string):
    if input_string == "fasting":
        return 0
    else:
        return 1

In [13]:
func_float = np.vectorize(to_float)
y_test_168_float = func_float(y_test_168)
y_test_bahai_float = func_float(y_test_bahai)

In [14]:
X_pred_parofastin_168.to_csv("evaluation/X_pred_parofastin_168.csv", sep='\t')
X_pred_parofastin_bahai.to_csv("evaluation/X_pred_parofastin_bahai.csv", sep='\t')
np.save('evaluation/y_pred_parofastin_168.npy', y_test_168_float)
np.save('evaluation/y_pred_parofastin_bahai.npy', y_test_bahai_float)

In [162]:
X_pred_parofastin_168 = pd.read_csv('X_pred_parofastin_168.csv', sep='\t')
X_pred_parofastin_bahai = pd.read_csv('X_pred_parofastin_bahai.csv', sep='\t')

y_test_168_float = np.load('evaluation/y_pred_parofastin_168.npy')
y_test_bahai_float = np.load('evaluation/y_pred_parofastin_bahai.npy')

In [23]:
from datetime import datetime


X_pred_parofastin_168.time = pd.to_datetime(X_pred_parofastin_168.time)
X_pred_parofastin_bahai.time = pd.to_datetime(X_pred_parofastin_bahai.time)

X_pred_parofastin_168_ml = X_pred_parofastin_168.drop(["time"], axis=1)
X_pred_parofastin_bahai_ml = X_pred_parofastin_bahai.drop(["time"], axis=1)


### Preprocessing for TSC features

In [3]:
from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn import generators

In [164]:

window_size = 5

x_list_pred_168 = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_168 = []
counter = 1

for i in range(0, X_pred_parofastin_168.shape[0], 1):

    time = X_pred_parofastin_168['time'].values[i]
    cgm_interdaycv = X_pred_parofastin_168['cgm_interdaycv'].values[i]
    J_index = X_pred_parofastin_168['J_index'].values[i]
    maximum = X_pred_parofastin_168['maximum'].values[i]


    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(cgm_interdaycv)
    x_list_pred_tsc_3rd.append(J_index)
    x_list_pred_tsc_3rd.append(maximum)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_168.append(y_test_168_float[i])

        x_list_pred_168.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_pred_168 = to_time_series_dataset(x_list_pred_168)

In [37]:
from datetime import datetime


x_list_pred_bahai = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_bahai = []
counter = 1

for i in range(0, X_pred_parofastin_bahai.shape[0], 1):

    time = X_pred_parofastin_bahai['time'].values[i]
    cgm_interdaycv = X_pred_parofastin_bahai['cgm_interdaycv'].values[i]
    J_index = X_pred_parofastin_bahai['J_index'].values[i]
    maximum = X_pred_parofastin_bahai['maximum'].values[i]


    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(cgm_interdaycv)
    x_list_pred_tsc_3rd.append(J_index)
    x_list_pred_tsc_3rd.append(maximum)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_bahai.append(y_test_bahai_float[i])

        x_list_pred_bahai.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_pred_bahai = to_time_series_dataset(x_list_pred_bahai)
    

In [165]:
x_list_pred_168.shape

(2172, 5, 3)

In [40]:
len(pred_labels_tsc_168)

2172

### Preprocessing for TSC raw

In [45]:
map_dict = {"non-fasting": 1, "fasting":0}
parofastin_168["fasting_state"] = parofastin_168["fasting_state"].map(map_dict)
parofastin_bahai["fasting_state"] = parofastin_bahai["fasting_state"].map(map_dict)

In [46]:
y_168_raw_float = parofastin_168["fasting_state"]
y_bahai_raw_float = parofastin_bahai["fasting_state"]

y_168_raw_float = np.array(y_168_raw_float)
y_bahai_raw_float = np.array(y_bahai_raw_float)

In [47]:
from datetime import datetime


parofastin_168.time = pd.to_datetime(parofastin_168.time)


x_list_pred_168_raw = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_168_raw = []
counter = 1

for i in range(0, parofastin_168.shape[0], 1):

    gl = parofastin_168['gl'].values[i]

    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(gl)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_168_raw.append(y_168_raw_float[i])

        x_list_pred_168_raw.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_pred_168_raw = to_time_series_dataset(x_list_pred_168_raw)

In [48]:
parofastin_bahai.time = pd.to_datetime(parofastin_bahai.time)


x_list_pred_bahai_raw = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_bahai_raw = []
counter = 1

for i in range(0, parofastin_bahai.shape[0], 1):

    gl = parofastin_bahai['gl'].values[i]

    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(gl)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_bahai_raw.append(y_bahai_raw_float[i])

        x_list_pred_bahai_raw.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_pred_bahai_raw = to_time_series_dataset(x_list_pred_bahai_raw)

In [49]:
x_list_pred_bahai_raw.shape

(2268, 5, 1)

In [50]:
len(pred_labels_tsc_bahai_raw)

2268

In [4]:
from numpy.lib.stride_tricks import sliding_window_view


def smoothing(y_pred):
    
    outcomes = np.array([])
    last_four_values = y_pred[-4:]
    
    y_pred_windowed = sliding_window_view((y_pred), window_shape = 5)

    i = 0
    for window in y_pred_windowed:
        pred_one = np.count_nonzero(window == 1)
        pred_zero = 5 - pred_one
        if pred_one > pred_zero:
            outcome = int(1)
            outcomes = np.append(outcomes, outcome)
        else:
            outcome = int(0)
            outcomes = np.append(outcomes, outcome)
    
    outcomes = np.append(outcomes, last_four_values)
            
    
    return outcomes

## Testing data on models

### ML Models

In [29]:
ml_types = ["ML_DT", "ML_KNN", "ML_SDG", "ML_LR", "ML_MLP", "ML_RF", "ML_SVM"]
folders = ["gl\\"]

In [30]:
start = "path_model"
end = "_adapt.pkl"

In [31]:
df_evaluation_parofastin_168 = pd.DataFrame(columns=['model','used_data','accuracy','smoothed_accuracy'])
df_evaluation_parofastin_bahai = pd.DataFrame(columns=['model','used_data','accuracy','smoothed_accuracy'])


import pickle

for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_168 = model.predict(X_pred_parofastin_168_ml)
        acc_16_8 = accuracy_score(y_test_168_float, y_pred_168)
        y_pred_bahai = model.predict(X_pred_parofastin_bahai_ml)
        acc_bahai = accuracy_score(y_test_bahai_float, y_pred_bahai)
        
        y_pred_168_smoothed = smoothing(y_pred_168)
        acc_16_8_smoothed = accuracy_score(y_test_168_float, y_pred_168_smoothed)
        y_pred_bahai_smoothed = smoothing(y_pred_bahai)
        acc_bahai_smoothed = accuracy_score(y_test_bahai_float, y_pred_bahai_smoothed)
        
        df_evaluation_parofastin_168 = df_evaluation_parofastin_168.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_16_8, "smoothed_accuracy": acc_16_8_smoothed}, ignore_index=True)
        df_evaluation_parofastin_bahai = df_evaluation_parofastin_bahai.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_bahai, "smoothed_accuracy": acc_bahai_smoothed}, ignore_index=True)


In [33]:
df_evaluation_parofastin_bahai

,model,used_data,accuracy,smoothed_accuracy
0,ML_DT,gl\,0.672399,0.712698
1,ML_KNN,gl\,0.597178,0.644797
2,ML_SDG,gl\,0.421781,0.399118
3,ML_LR,gl\,0.683069,0.711023
4,ML_MLP,gl\,0.693122,0.728748
5,ML_RF,gl\,0.688183,0.726808
6,ML_SVM,gl\,0.699118,0.726014


### TSC models

In [166]:
ml_types = ["CLF_TSC", "KNN_TSC"]
folders = ["gl\\"]
start = "path_model
end = ".pkl"

In [ ]:
for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_168 = model.predict(x_list_pred_168)
        acc_16_8 = accuracy_score(pred_labels_tsc_168, y_pred_168)
        y_pred_bahai = model.predict(x_list_pred_bahai)
        acc_bahai = accuracy_score(pred_labels_tsc_bahai, y_pred_bahai)
        
        y_pred_168_smoothed = smoothing(y_pred_168)
        acc_16_8_smoothed = accuracy_score(pred_labels_tsc_168, y_pred_168_smoothed)
        y_pred_bahai_smoothed = smoothing(y_pred_bahai)
        acc_bahai_smoothed = accuracy_score(pred_labels_tsc_bahai, y_pred_bahai_smoothed)
        
        df_evaluation_parofastin_168 = df_evaluation_parofastin_168.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_16_8, "smoothed_accuracy": acc_16_8_smoothed}, ignore_index=True)
        df_evaluation_parofastin_bahai = df_evaluation_parofastin_bahai.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_bahai, "smoothed_accuracy": acc_bahai_smoothed}, ignore_index=True)

### TSC models raw

In [52]:
ml_types = ["CLF_TSC", "KNN_TSC"]
folders = ["gl\\"]
start = "path_model"
end = ".pkl"

In [54]:
for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_168_raw = model.predict(x_list_pred_168_raw)
        acc_16_8_raw = accuracy_score(pred_labels_tsc_168_raw, y_pred_168_raw)
        y_pred_bahai_raw = model.predict(x_list_pred_bahai_raw)
        acc_bahai_raw = accuracy_score(pred_labels_tsc_bahai_raw, y_pred_bahai_raw)
        
        y_pred_168_raw_smoothed = smoothing(y_pred_168_raw)
        acc_16_8_raw_smoothed = accuracy_score(pred_labels_tsc_168_raw, y_pred_168_raw_smoothed)
        y_pred_bahai_raw_smoothed = smoothing(y_pred_bahai_raw)
        acc_bahai_raw_smoothed = accuracy_score(pred_labels_tsc_bahai_raw, y_pred_bahai_raw_smoothed)
        
        df_evaluation_parofastin_168 = df_evaluation_parofastin_168.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_16_8_raw, "smoothed_accuracy": acc_16_8_raw_smoothed}, ignore_index=True)
        df_evaluation_parofastin_bahai = df_evaluation_parofastin_bahai.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_bahai_raw, "smoothed_accuracy": acc_bahai_raw_smoothed}, ignore_index=True)

In [67]:
df_evaluation_parofastin_168.to_csv("evaluation/df_evaluation_parofastin_168.csv", sep='\t')
df_evaluation_parofastin_bahai.to_csv("evaluation/df_evaluation_parofastin_bahai.csv", sep='\t')

In [69]:
df_evaluation_parofastin_bahai

,model,used_data,accuracy,smoothed_accuracy
4,ML_MLP,gl\,0.693122,0.728748
5,ML_RF,gl\,0.688183,0.726808
6,ML_SVM,gl\,0.699118,0.726014
11,Hutchison,gl and nutrition,0.691,0.7178
0,ML_DT,gl\,0.672399,0.712698
3,ML_LR,gl\,0.683069,0.711023
7,CLF_TSC,gl\,0.71164,0.702822
9,CLF_TSC,gl\,0.685185,0.690476
10,KNN_TSC,gl\,0.650794,0.656085
8,KNN_TSC,gl\,0.676808,0.653439


In [66]:
df_evaluation_parofastin_168


,model,used_data,accuracy,smoothed_accuracy
4,ML_MLP,gl\,0.718494,0.7358
3,ML_LR,gl\,0.716745,0.734788
7,CLF_TSC,gl\,0.720074,0.734346
6,ML_SVM,gl\,0.713615,0.730553
9,CLF_TSC,gl\,0.704556,0.724804
5,ML_RF,gl\,0.681027,0.71472
0,ML_DT,gl\,0.636841,0.667863
1,ML_KNN,gl\,0.614563,0.657277
8,KNN_TSC,gl\,0.634899,0.640424
11,Hutchison,gl and nutrition,0.6121,0.6389


In [58]:
new_row_1 = {'model':'Hutchison', 'used_data':"gl and nutrition", 'accuracy':0.691, 'smoothed_accuracy':0.7178}
new_row_2 = {'model':'Hutchison', 'used_data':"gl and nutrition", 'accuracy':0.6121, 'smoothed_accuracy':0.6389}


df_evaluation_parofastin_bahai = df_evaluation_parofastin_bahai.append(new_row_1, ignore_index=True)
df_evaluation_parofastin_168 = df_evaluation_parofastin_168.append(new_row_2, ignore_index=True)

In [68]:
df_evaluation_parofastin_bahai = df_evaluation_parofastin_bahai.sort_values(by=['smoothed_accuracy'], ascending=False)
df_evaluation_parofastin_168 = df_evaluation_parofastin_168.sort_values(by=['smoothed_accuracy'], ascending=False)


 # Evaluation of simglucosedata

In [78]:
simglucose = pd.read_csv('path_model', sep='\t')

In [79]:
simglucose['Time'] = pd.to_datetime(simglucose['Time'], format="%Y-%m-%d %H:%M:%S")
simglucose['Day'] = simglucose["Time"].dt.date
simglucose['gl'] = pd.to_numeric(simglucose['CGM'])
simglucose["Glucose"] = simglucose["gl"]


In [80]:
simglucose['Time'] = pd.to_datetime(simglucose['Time'])
simglucose = simglucose.set_index('Time')

In [82]:
simglucose = simglucose.resample('15T').mean()


In [84]:
simglucose

,Unnamed: 0.1,Unnamed: 0,CGM,id,gl,Glucose
Time,,,,,,
2022-12-12 00:00:00,2647.5,2.0,144.206927,2,144.206927,144.206927
2022-12-12 00:15:00,2652.5,7.0,140.191103,2,140.191103,140.191103
2022-12-12 00:30:00,2657.5,12.0,137.322261,2,137.322261,137.322261
2022-12-12 00:45:00,2662.5,17.0,140.407369,2,140.407369,140.407369
2022-12-12 01:00:00,2667.5,22.0,142.934589,2,142.934589,142.934589
...,...,...,...,...,...,...
2022-12-12 23:00:00,3107.5,462.0,165.667827,2,165.667827,165.667827
2022-12-12 23:15:00,3112.5,467.0,151.076417,2,151.076417,151.076417
2022-12-12 23:30:00,3117.5,472.0,138.442153,2,138.442153,138.442153


In [85]:
simglucose.to_csv("simglucose_data_resampled.csv", sep='\t')


In [8]:
x_list_simglucose = []
train_labels_simglucose = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, simglucose.shape[0] - window_size, step_size):
    time = simglucose['Time'].values[i: i + window_size]
    times = simglucose['Day'].values[i: i + window_size]
    gls = simglucose['Glucose'].values[i: i + window_size]

    label = stats.mode(simglucose['fasting_state'][i: i + window_size])[0][0]

    df_slice = pd.DataFrame(time, columns=['Time'])
    df_slice["Day"] = times
    df_slice["Glucose"] = gls   

    x_list_simglucose.append(df_slice)

    train_labels_simglucose.append(label)

In [10]:
X_pred_simglucose = feat_statistical_measures_gl(x_list_simglucose)


In [11]:
y_test_simglucose = np.array(train_labels_simglucose)

def to_float(input_string):
    if input_string == "fasting":
        return 0
    else:
        return 1

In [12]:
func_float = np.vectorize(to_float)
y_test_simglucose = func_float(y_test_simglucose)


In [ ]:
from datetime import datetime


X_pred_simglucose.time = pd.to_datetime(X_pred_simglucose.time)

X_pred_simglucose = X_pred_simglucose.drop(["time"], axis=1)


In [42]:
X_pred_simglucose

,mean,median,minimum,maximum,first_quartile,third_quartile,LBGI,HBGI,ADRR,GMI,J_index,eA1c,interdaysd,cgm_interdaycv,cgm_TOR,cgm_TIR,cgm_POR
0,151.846281,152.028551,149.839030,153.671262,150.933790,152.849906,0.000000,3.139627,3.392696,6.942163,23.536494,6.917989,1.569802,1.033810,30,15,66.666667
1,149.674039,149.839030,147.154536,152.028551,148.496783,150.933790,0.000000,2.846435,3.162197,6.890203,23.002959,6.842301,1.993225,1.331711,30,15,66.666667
2,147.115926,147.154536,144.354213,149.839030,145.754375,148.496783,0.000000,2.514187,2.863988,6.829013,22.306994,6.753168,2.239333,1.522156,30,15,66.666667
3,144.441983,144.354213,141.817201,147.154536,143.085707,145.754375,0.000000,2.182074,2.513120,6.765052,21.497960,6.659999,2.179842,1.509147,30,15,66.666667
4,142.031352,141.817201,139.922642,144.354213,140.869921,143.085707,0.000000,1.896535,2.165454,6.707390,20.691919,6.576005,1.815507,1.278244,30,15,66.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4803,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4804,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4805,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000


## Testing on models

### ML models

In [43]:
ml_types = ["ML_DT", "ML_KNN", "ML_SDG", "ML_LR", "ML_MLP", "ML_RF", "ML_SVM"]
folders = ["gl\\"]

In [44]:
start = "model_path"
end = "_adapt.pkl"

In [45]:
scaler = pickle.load(open("ML_scaler_adapt.pkl", 'rb'))


In [46]:
X_pred_simglucose = scaler.transform(X_pred_simglucose)


In [47]:
df_evaluation_simglucose = pd.DataFrame(columns=['model','used_data','accuracy','smoothed_accuracy'])


import pickle

for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_simglucose = model.predict(X_pred_simglucose)
        acc_simglucose = accuracy_score(y_test_simglucose, y_pred_simglucose)
        
        y_pred_simglucose_smoothed = smoothing(y_pred_simglucose)
        acc_simglucose_smoothed = accuracy_score(y_test_simglucose, y_pred_simglucose_smoothed)

        
        df_evaluation_simglucose = df_evaluation_simglucose.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_simglucose, "smoothed_accuracy": acc_simglucose_smoothed}, ignore_index=True)


In [125]:
print(np.unique(y_pred_simglucose))

[1]


In [48]:
df_evaluation_simglucose

,model,used_data,accuracy,smoothed_accuracy
0,ML_DT,gl\,0.710214,0.716871
1,ML_KNN,gl\,0.756605,0.756605
2,ML_SDG,gl\,0.636156,0.656126
3,ML_LR,gl\,0.756605,0.756605
4,ML_MLP,gl\,0.243395,0.243395
5,ML_RF,gl\,0.756605,0.756605
6,ML_SVM,gl\,0.243395,0.243395


### TSC models

In [153]:
X_pred_simglucose

,time,mean,median,minimum,maximum,first_quartile,third_quartile,LBGI,HBGI,ADRR,GMI,J_index,eA1c,interdaysd,cgm_interdaycv,cgm_TOR,cgm_TIR,cgm_POR
0,2022-12-12 00:03:00,151.846281,152.028551,149.839030,153.671262,150.933790,152.849906,0.000000,3.139627,3.392696,6.942163,23.536494,6.917989,1.569802,1.033810,30,15,66.666667
1,2022-12-12 00:06:00,149.674039,149.839030,147.154536,152.028551,148.496783,150.933790,0.000000,2.846435,3.162197,6.890203,23.002959,6.842301,1.993225,1.331711,30,15,66.666667
2,2022-12-12 00:09:00,147.115926,147.154536,144.354213,149.839030,145.754375,148.496783,0.000000,2.514187,2.863988,6.829013,22.306994,6.753168,2.239333,1.522156,30,15,66.666667
3,2022-12-12 00:12:00,144.441983,144.354213,141.817201,147.154536,143.085707,145.754375,0.000000,2.182074,2.513120,6.765052,21.497960,6.659999,2.179842,1.509147,30,15,66.666667
4,2022-12-12 00:15:00,142.031352,141.817201,139.922642,144.354213,140.869921,143.085707,0.000000,1.896535,2.165454,6.707390,20.691919,6.576005,1.815507,1.278244,30,15,66.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,2022-12-12 23:42:00,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4803,2022-12-12 23:45:00,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4804,2022-12-12 23:48:00,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000
4805,2022-12-12 23:51:00,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,38.200821,0.000000,38.200821,4.242880,1.521000,2.986063,0.000000,0.000000,45,45,100.000000


In [13]:

window_size = 5

x_list_simglucose = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_simglucose = []
counter = 1

for i in range(0, X_pred_simglucose.shape[0], 1):

    time = X_pred_simglucose['time'].values[i]
    cgm_interdaycv = X_pred_simglucose['cgm_interdaycv'].values[i]
    J_index = X_pred_simglucose['J_index'].values[i]
    maximum = X_pred_simglucose['maximum'].values[i]


    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(cgm_interdaycv)
    x_list_pred_tsc_3rd.append(J_index)
    x_list_pred_tsc_3rd.append(maximum)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_simglucose.append(y_test_simglucose[i])

        x_list_simglucose.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_simglucose = to_time_series_dataset(x_list_simglucose)

In [58]:
len(pred_labels_tsc_simglucose)

961

In [59]:
x_list_simglucose.shape

(961, 5, 3)

### Test

In [14]:
ml_types = ["CLF_TSC", "KNN_TSC"]
folders = ["gl\\"]
start = "models_TSC"
end = ".pkl"

In [ ]:
for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_simglucose = model.predict(x_list_simglucose)
        acc_simglucose = accuracy_score(pred_labels_tsc_simglucose, y_pred_simglucose)

        
        y_pred_simglucose_smoothed = smoothing(y_pred_simglucose)
        acc_simglucose_smoothed = accuracy_score(pred_labels_tsc_simglucose, y_pred_simglucose_smoothed)

        
        df_evaluation_simglucose = df_evaluation_simglucose.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_simglucose, "smoothed_accuracy": acc_simglucose_smoothed}, ignore_index=True)


In [73]:
df_evaluation_simglucose

,model,used_data,accuracy,smoothed_accuracy
0,ML_DT,gl\,0.710214,0.716871
1,ML_KNN,gl\,0.756605,0.756605
2,ML_SDG,gl\,0.636156,0.656126
3,ML_LR,gl\,0.756605,0.756605
4,ML_MLP,gl\,0.243395,0.243395
5,ML_RF,gl\,0.756605,0.756605
6,ML_SVM,gl\,0.243395,0.243395
7,CLF_TSC,gl\,0.322581,0.355879
8,KNN_TSC,gl\,0.479709,0.545265
9,CLF_TSC,gl\,0.321206,0.355509


### Raw

In [65]:
simglucose

,Unnamed: 0.1,Unnamed: 0,Time,CGM,id,fasting_state,Day,gl,Glucose
0,0,0,2022-12-12 00:00:00,153.671262,1,fasting,2022-12-12,153.671262,153.671262
1,1,1,2022-12-12 00:03:00,152.028551,1,fasting,2022-12-12,152.028551,152.028551
2,2,2,2022-12-12 00:06:00,149.839030,1,fasting,2022-12-12,149.839030,149.839030
3,3,3,2022-12-12 00:09:00,147.154536,1,fasting,2022-12-12,147.154536,147.154536
4,4,4,2022-12-12 00:12:00,144.354213,1,fasting,2022-12-12,144.354213,144.354213
...,...,...,...,...,...,...,...,...,...
4805,4805,476,2022-12-12 23:48:00,39.000000,10,fasting,2022-12-12,39.000000,39.000000
4806,4806,477,2022-12-12 23:51:00,39.000000,10,fasting,2022-12-12,39.000000,39.000000
4807,4807,478,2022-12-12 23:54:00,39.000000,10,fasting,2022-12-12,39.000000,39.000000
4808,4808,479,2022-12-12 23:57:00,39.000000,10,fasting,2022-12-12,39.000000,39.000000


In [18]:
map_dict = {"non-fasting": 1, "fasting":0}
simglucose["fasting_state"] = simglucose["fasting_state"].map(map_dict)


In [19]:
y_simglucose_raw_float = simglucose["fasting_state"]

y_simglucose_raw_float = np.array(y_simglucose_raw_float)


In [20]:
y_simglucose_raw_float


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:

window_size = 5

x_list_simglucose = []
x_list_pred_tsc_2nd = []
pred_labels_tsc_simglucose = []
counter = 1

for i in range(0, simglucose.shape[0], 1):

    gl = simglucose['gl'].values[i]


    x_list_pred_tsc_3rd = []
    x_list_pred_tsc_3rd.append(gl)

    x_list_pred_tsc_2nd.append(x_list_pred_tsc_3rd)

    if counter >= window_size:
        pred_labels_tsc_simglucose.append(y_simglucose_raw_float[i])

        x_list_simglucose.append(x_list_pred_tsc_2nd)
        x_list_pred_tsc_2nd = []
        counter = 0

    counter += 1

x_list_simglucose = to_time_series_dataset(x_list_simglucose)

In [22]:
ml_types = ["KNN_TSC"]
folders = ["gl\\"]
start = "models_raw_TSC"
end = "_scaled.pkl"

In [25]:
df_evaluation_simglucose = pd.DataFrame(columns=['model','used_data','accuracy','smoothed_accuracy'])


for ml_type in ml_types:
    for folder in folders:
        
        path = start + folder + ml_type + end

        with open(path, "rb") as f:
            model = pickle.load(f)
            
        y_pred_simglucose = model.predict(x_list_simglucose)
        acc_simglucose = accuracy_score(pred_labels_tsc_simglucose, y_pred_simglucose)

        
        y_pred_simglucose_smoothed = smoothing(y_pred_simglucose)
        acc_simglucose_smoothed = accuracy_score(pred_labels_tsc_simglucose, y_pred_simglucose_smoothed)

        
        df_evaluation_simglucose = df_evaluation_simglucose.append({'model': ml_type, 'used_data':folder, 'accuracy': acc_simglucose, "smoothed_accuracy": acc_simglucose_smoothed}, ignore_index=True)


In [26]:
df_evaluation_simglucose

,model,used_data,accuracy,smoothed_accuracy
0,KNN_TSC,gl\,0.243243,0.243243


In [72]:
df_evaluation_simglucose.to_csv("evaluation/df_evaluation_simglucose.csv", sep='\t')
